In [1]:
import os
import sys
import numpy as np
import stingray
import scipy.io
from astropy.io import fits
import matplotlib.pyplot as plt
from nicer_fuc import *

/home/pl/anaconda3/lib/python3.11/site-packages/stingray/utils.py:35: UserWarning: pyfftw not installed. Using standard scipy fft
  warnings.warn("pyfftw not installed. Using standard scipy fft")


In [2]:
cwd = os.getcwd()
obs_list = get_obs(cwd)

In [3]:
for obs in obs_list:
    print_log(obs)
    obs = obs[0:10]

    lc_file = f'ni{obs}mpu7_sr1s.lc'
    lc_path = os.path.join(cwd,obs,lc_file)   

    with fits.open(lc_path) as data:

        #查看头文件信息
        header = data[1].header
        print(header)

        times = np.array(data[1].data['TIME'])
        rates = np.array(data[1].data['RATE'])

        src_name = data[1].header['object']
        print(src_name)

        ra,dec = get_ra_dec(src_name)
        print(ra,dec)

        timezero = data[1].header['MJDREFI'] + data[1].header['MJDREFF'] + data[1].header['TSTART'] / 24 / 3600
        print(timezero) #MJD时间

        t_zero = data[1].header['TIMEZERO']   #xspec能识别的时间戳
        print(t_zero)

        print()
        print(times[0])

        mjd_refi = data[1].header['MJDREFI']
        mjd_reff = data[1].header['MJDREFF']
        print(mjd_refi)

1050080126
XTENSION= 'BINTABLE'           / binary table extension                         BITPIX  =                    8 / 8-bit bytes                                    NAXIS   =                    2 / 2-dimensional binary table                     NAXIS1  =                   20 / width of table in bytes                        NAXIS2  =                 1739 / Number of bins                                 PCOUNT  =                    0 / size of special data area                      GCOUNT  =                    1 / one data group (required keyword)              TFIELDS =                    4 / number of fields in each row                   TTYPE1  = 'TIME    '           / label for field   1                            TFORM1  = 'D       '           / data format of field: 8-byte DOUBLE            TUNIT1  = 's       '           / physical unit of field                         TTYPE2  = 'RATE    '           / label for field   2                            TFORM2  = 'E       '         

In [5]:
from astropy.io import fits
import os

from astropy.io import fits
import os

def fits2dict(filename, par_list, ext):
    """
    从FITS文件的指定扩展中读取指定列或头文件键的数据并返回字典。

    参数:
    filename (str): FITS文件名。
    par_list (list of str): 要提取的列名或头文件键名列表。
    ext (int): FITS文件的扩展号,默认将header全部添加。
        0 -> header, 如 'OBJECT', 'MJDREFI', 'MJDREFF', 'TIMEZERO' 等。
        1 -> EVENTS, 如 'TIME', 'RATE', 'ERROR' 等。
        2 -> FPM_SEL。
        3 -> GTI。
        4 -> PPS_TREND。

    返回:
    dict: 包含指定扩展数据的字典。
    """
    with fits.open(filename) as hdul:
        results = {}

        header = hdul[0].header
        for key in par_list:
            if key.upper() in header:
                results[key] = header[key.upper()]

        data = hdul[ext].data
        if data is not None:
            for key in par_list:
                if key in data.names:
                    results[key] = data[key]
        else:
            raise ValueError(f"扩展 {ext} 不包含数据表。")

    return results


# 使用示例
# gtis = fits.open(eventfile)[3].data
cwd = os.getcwd()
obs = '1050080126'

lc_file = f'ni{obs}mpu7_sr1s.lc'
lc_path = os.path.join(cwd, obs, lc_file)

par_list = ['TIME', 'RATE', 'ERROR', 'object', 'MJDREFI', 'MJDREFF', 'TIMEZERO']
# par_list = ['TIME', 'RATE', 'ERROR']
data_dict = fits2dict(lc_path,par_list,1)
print(data_dict['TIME'],data_dict['ERROR'] ,data_dict['object'],data_dict['MJDREFI'] )




[0.0000e+00 1.0000e+00 2.0000e+00 ... 1.6925e+04 1.6926e+04 1.6927e+04] [25.690466 25.495098 24.083189 ... 26.1916   25.65151  25.13961 ] 4U_1636-536 56658
